In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.metrics import mean_absolute_error

# Load the dataset
file_path = 'Merged_GNSS_and_SNR_Data.csv'
gnss_data = pd.read_csv(file_path)

# Define numerical and categorical columns
numerical_cols = ['Phase Residual (m)', 'Pseudorange Residual (m)', 
                  'STD of Pseudorange L1 (m)', 'STD of Pseudorange L2 (m)', 
                  'Elevation (¡Æ)', 'Azimuth (¡Æ)', 'SNR(dBHz)']

categorical_cols = ['PRN', 'Signal Type 1', 'Signal Type 2']

# Drop irrelevant or target-related columns from input features
X = gnss_data.drop(columns=['TIME', 'L1 MP(m)', 'C1 Code', 'C2 Code', 'DSI'])


# Target variable (multipath effect on L1 MP)
y = gnss_data['L1 MP(m)']

# Preprocessing for numerical and categorical data
preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), numerical_cols),
        ('cat', OneHotEncoder(handle_unknown='ignore'), categorical_cols)])

# Split the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Fit the preprocessor on the training data and transform both train and test sets
X_train_preprocessed = preprocessor.fit_transform(X_train)
X_test_preprocessed = preprocessor.transform(X_test)

# Convert sparse matrices to dense
X_train_preprocessed = X_train_preprocessed.toarray()  # Convert sparse matrix to dense
X_test_preprocessed = X_test_preprocessed.toarray()    # Convert sparse matrix to dense

In [4]:
import tensorflow as tf
from keras import layers, models
from sklearn.metrics import mean_absolute_error

# Define the Fully Connected Neural Network (FCNN) model
fcnn_model = models.Sequential()

# Input layer
fcnn_model.add(layers.InputLayer(input_shape=(X_train_preprocessed.shape[1],)))

# Fully connected hidden layer 1
fcnn_model.add(layers.Dense(128, activation='relu'))

# Fully connected hidden layer 2
fcnn_model.add(layers.Dense(64, activation='relu'))

# Output layer (regression task for predicting L1 MP)
fcnn_model.add(layers.Dense(1, activation='linear'))

# Compile the model
fcnn_model.compile(optimizer='adam', loss='mean_squared_error', metrics=['mean_absolute_error'])

# Model summary
fcnn_model.summary()

/opt/anaconda3/envs/sklearn-env/lib/python3.12/site-packages/keras/src/layers/core/input_layer.py:26: UserWarning: Argument `input_shape` is deprecated. Use `shape` instead.
  warnings.warn(


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 128)            │         5,248 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 64)             │         8,256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 1)              │            65 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 13,569 (53.00 KB)

 Trainable params: 13,569 (53.00 KB)

 Non-trainable params: 0 (0.00 B)

In [5]:
# Train the FCNN model
fcnn_history = fcnn_model.fit(X_train_preprocessed, y_train, epochs=10, batch_size=32, validation_data=(X_test_preprocessed, y_test))


Epoch 1/10
9388/9388 ━━━━━━━━━━━━━━━━━━━━ 3s 298us/step - loss: 0.0304 - mean_absolute_error: 0.1199 - val_loss: 0.0266 - val_mean_absolute_error: 0.1130
Epoch 2/10
9388/9388 ━━━━━━━━━━━━━━━━━━━━ 3s 291us/step - loss: 0.0258 - mean_absolute_error: 0.1118 - val_loss: 0.0256 - val_mean_absolute_error: 0.1115
Epoch 3/10
9388/9388 ━━━━━━━━━━━━━━━━━━━━ 3s 293us/step - loss: 0.0267 - mean_absolute_error: 0.1108 - val_loss: 0.0256 - val_mean_absolute_error: 0.1102
Epoch 4/10
9388/9388 ━━━━━━━━━━━━━━━━━━━━ 3s 295us/step - loss: 0.0246 - mean_absolute_error: 0.1091 - val_loss: 0.0250 - val_mean_absolute_error: 0.1096
Epoch 5/10
9388/9388 ━━━━━━━━━━━━━━━━━━━━ 3s 295us/step - loss: 0.0245 - mean_absolute_error: 0.1087 - val_loss: 0.0246 - val_mean_absolute_error: 0.1087
Epoch 6/10
9388/9388 ━━━━━━━━━━━━━━━━━━━━ 3s 294us/step - loss: 0.0245 - mean_absolute_error: 0.1083 - val_loss: 0.0244 - val_mean_absolute_error: 0.1084
Epoch 7/10
9388/9388 ━━━━━━━━━━━━━━━━━━━━ 3s 297us/step - loss: 0.0247 - mea

In [6]:
# Predict using the FCNN model
y_pred_fcnn = fcnn_model.predict(X_test_preprocessed)

# Calculate MAE for the FCNN
fcnn_mae = mean_absolute_error(y_test, y_pred_fcnn)
print(f"Fully Connected Neural Network MAE: {fcnn_mae}")

2347/2347 ━━━━━━━━━━━━━━━━━━━━ 0s 177us/step
Fully Connected Neural Network MAE: 0.10655459468799028
